<a href="https://colab.research.google.com/github/harperd17/energy_volatility_prediction/blob/main/notebooks/USB_ingestion_all_years.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gather Data 2001 - 2018 from SUSB for back fill of electric accounts

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import requests
import xlrd
import urllib
from google.colab import drive

!pip install us
import us

     |████████████████████████████████| 132 kB 3.9 MB/s 
  Created wheel for us: filename=us-2.0.2-py3-none-any.whl size=11942 sha256=6b0e1a8a5588805e936873021d6f6ae215ca627b525ce21cdefb1bcf4b65c6d1
  Stored in directory: /root/.cache/pip/wheels/ca/6b/11/cda9ea2438f721330a35c9a2c8e34b4aedcd34c89af48a4d00
  Created wheel for jellyfish: filename=jellyfish-0.6.1-cp37-cp37m-linux_x86_64.whl size=72185 sha256=6cd4db17bc71b5bec0a88414995e811768df37bb91981834174971ce864e988c
  Stored in directory: /root/.cache/pip/wheels/a1/99/51/7de469e37cd1b3c763c24394e1ebf1baa2d79e094bf346cf80
Successfully built us jellyfish


In [ ]:
drive.mount('/content/drive/')

## Get the data for each year

In [ ]:
# this contains the information needed to properly gather the data for each year - this is necessary because the data changes so much year to year
metadata = {
    '2001':{
        'link': 'https://www2.census.gov/programs-surveys/susb/tables/2001/state_naicssector_2001.xls',
        'header row':[7],
        'columns':['STATE', 'CODE', 'DESCRIPTION', 'DATA TYPE', 'TOTAL', '0',
                    '1-4', '5-9', '10-19', '<20', '20-99', '100-499', '<500', '500+']
    },
    '2002':{
        'link':'https://www2.census.gov/programs-surveys/susb/tables/2002/state_naicssector_2002.xls',
        'header row':[5,6],
        'columns':['STATE','CODE','DESCRIPTION', 'DATA TYPE', 'TOTAL', '0',
                    '1-4', '5-9', '10-19', '<20', '20-99', '100-499', '<500', '500+']
    },
    '2003':{
        'link':'https://www2.census.gov/programs-surveys/susb/tables/2003/state_naicssector_2003.xls',
        'header row':[7],
        'columns':['STATE','CODE','DESCRIPTION', 'DATA TYPE', 'TOTAL', '0',
                    '1-4', '5-9', '10-19', '<20', '20-99', '100-499', '<500', '500+']
    },
    '2004':{
        'link':'https://www2.census.gov/programs-surveys/susb/tables/2004/state_naicssector_2004.xls',
        'header row':[7],
        'columns':['STATE','CODE','DESCRIPTION', 'DATA TYPE', 'TOTAL', '0',
                    '1-4', '5-9', '10-19', '<20', '20-99', '100-499', '<500', '500+']
    },
    '2005':{
        'link':'https://www2.census.gov/programs-surveys/susb/tables/2005/state_naicssector_2005.xls',
        'header row':[7],
        'columns':['STATE', 'CODE', 'DESCRIPTION', 'DATA TYPE', 'TOTAL', '0-4',
                    '5-9', '10-19', '<20', '20-99', '100-499', '<500', '500+']
    },
    '2006':{
        'link':'https://www2.census.gov/programs-surveys/susb/tables/2006/state_naicssector_2006.xls',
        'header row':[7],
        'columns':['STATE','CODE','DESCRIPTION', 'DATA TYPE', 'TOTAL',
                    '0-4', '5-9', '10-19', '<20', '20-99', '100-499', '<500', '500+']
    },
    '2007':{
        'link':'https://www2.census.gov/programs-surveys/susb/tables/2007/state_naicssector_2007.xls',
        'header row':[5,6],
        'columns':['STATE', 'CODE', 'DESCRIPTION', 'SIZE','FIRMS', 'ESTABLISHMENTS',
                  'EMPLOYMENT', 'EMPLOYMENT RANGE FLAG','EMPLOYMENT NOISE FLAG', 'PAYROLL', 'PAYROLL NOISE FLAG', 'RECEIPTS','RECEIPTS NOISE FLAG'],
        'pivot':True
    },
    '2008':{
        'link':'https://www2.census.gov/programs-surveys/susb/tables/2008/state_naicssector_2008.xls',
        'header row':[5],
        'columns':['STATE CODE', 'STATE', 'CODE', 'DESCRIPTION', 'SIZE','FIRMS', 'ESTABLISHMENTS', 'EMPLOYMENT', 'RANGE FLAG',
                  'NOISE FLAG', '-1000', 'NOISE FLAG.1'],
        'pivot':True
    },
    '2009':{
        'link':'https://www2.census.gov/programs-surveys/susb/tables/2009/state_naicssector_2009.xls',
        'header row':[4],
        'columns':['STATE CODE', 'STATE', 'CODE','DESCRIPTION', 'SIZE', 'FIRMS', 'ESTABLISHMENTS', 'EMPLOYMENT',
                  'EMPLOYMENT RANGE FLAG', 'EMPLOYMENT NOISE FLAG', 'ANNUAL PAYROLL ($1,000)', 'ANNUAL PAYROLL NOISE FLAG'],
        'pivot':True
    },
    '2010':{
        'link':'https://www2.census.gov/programs-surveys/susb/tables/2010/state_naicssector_2010.xls',
        'header row':[4],
        'columns':['STATE CODE', 'STATE', 'CODE', 'DESCRIPTION', 'SIZE','FIRMS', 'ESTABLISHMENTS', 'EMPLOYMENT',
                'EMPLOYMENT RANGE FLAG', 'EMPLOYMENT NOISE FLAG','ANNUAL PAYROLL ($1,000)', 'ANNUAL PAYROLL NOISE FLAG'],
        'pivot':True
    },
    '2011':{
        'link':'https://www2.census.gov/programs-surveys/susb/tables/2011/state_naicssector_2011.xls',
        'header row':[4],
        'columns':['STATE CODE', 'STATE', 'CODE','DESCRIPTION', 'SIZE','FIRMS', 'ESTABLISHMENTS', 'EMPLOYMENT',
                  'EMPLOYMENT RANGE FLAG', 'EMPLOYMENT NOISE FLAG','ANNUAL PAYROLL ($1,000)', 'ANNUAL PAYROLL NOISE FLAG'],
       'pivot':True
    },
    '2012':{
        'link':'https://www2.census.gov/programs-surveys/susb/tables/2012/state_naicssector_2012.xls',
        'header row':[5],
        'columns':['STATE CODE', 'STATE', 'CODE','DESCRIPTION', 'SIZE', 'FIRMS', 'ESTABLISHMENTS', 'EMPLOYMENT',
                  'EMPLOYMENT RANGE FLAG', 'EMPLOYMENT NOISE FLAG','ANNUAL PAYROLL ($1,000)', 'ANNUAL PAYROLL NOISE FLAG', 'RECEIPTS      ($1,000)', 'RECEIPTS NOISE FLAG'],
        'pivot':True
    },
    '2013':{
        'link':'https://www2.census.gov/programs-surveys/susb/tables/2013/state_naicssector_2013.xlsx',
        'header row':[5],
        'columns':['STATE CODE', 'STATE', 'CODE','DESCRIPTION', 'SIZE', 'FIRMS', 'ESTABLISHMENTS', 'EMPLOYMENT',
                  'EMPLOYMENT RANGE FLAG', 'EMPLOYMENT NOISE FLAG', 'ANNUAL PAYROLL ($1,000)', 'ANNUAL PAYROLL NOISE FLAG'],
        'pivot':True
    },
    '2014':{
        'link':'https://www2.census.gov/programs-surveys/susb/tables/2014/state_naicssector_2014.xlsx',
        'header row':[5],
        'columns':['STATE CODE', 'STATE', 'CODE','DESCRIPTION', 'SIZE','FIRMS', 'ESTABLISHMENTS', 'EMPLOYMENT',
                'EMPLOYMENT RANGE FLAG', 'EMPLOYMENT NOISE FLAG','ANNUAL PAYROLL ($1,000)', 'ANNUAL PAYROLL NOISE FLAG'],
        'pivot':True
    },
    '2015':{
        'link':'https://www2.census.gov/programs-surveys/susb/tables/2015/state_naicssector_2015.xlsx',
        'header row':[5],
        'columns':['STATE CODE', 'STATE', 'CODE','DESCRIPTION', 'SIZE','FIRMS', 'ESTABLISHMENTS', 'EMPLOYMENT',
                  'EMPLOYMENT RANGE FLAG', 'EMPLOYMENT NOISE FLAG','ANNUAL PAYROLL ($1,000)', 'ANNUAL PAYROLL NOISE FLAG'],
        'pivot':True
    },
    '2016':{
        'link':'https://www2.census.gov/programs-surveys/susb/tables/2016/state_naicssector_2016.xlsx',
        'header row':[6],
        'columns':['STATE CODE', 'STATE', 'CODE','DESCRIPTION', 'SIZE','FIRMS', 'ESTABLISHMENTS', 'EMPLOYMENT',
                  'EMPLOYMENT RANGE FLAG', 'EMPLOYMENT NOISE FLAG','ANNUAL PAYROLL ($1,000)', 'ANNUAL PAYROLL NOISE FLAG'],
        'pivot':True
    },
    '2017':{
        'link':'https://www2.census.gov/programs-surveys/susb/tables/2017/us_state_6digitnaics_2017.xlsx',
        'header row':[2],
        'columns':['STATE CODE', 'STATE', 'CODE', 'DESCRIPTION','SIZE', 'FIRMS', 'ESTABLISHMENTS', 'EMPLOYMENT','Employment\nRange Flag', 'Employment Noise Flag',
                   'Annual Payroll\n($1,000)', 'Annual Payroll Noise Flag','Receipts\n($1,000)', 'Receipts\nNoise Flag'],
        'pivot':True
    },
    '2018':{
        'link':'https://www2.census.gov/programs-surveys/susb/tables/2018/us_state_6digitnaics_2018.xlsx',
        'header row':[2],
        'columns':['STATE CODE', 'STATE', 'CODE', 'DESCRIPTION', 'SIZE', 'FIRMS', 'ESTABLISHMENTS', 'EMPLOYMENT',
                  'Employment Noise Flag', 'Annual Payroll\n($1,000)','Annual Payroll Noise Flag'],
        'pivot':True
    }
}

In [ ]:
# go through each year and gather the data and append the df for each year to list
all_dfs_raw = []
dfs_to_keep_2001_2006 = []
dfs_to_keep_2007_2018 = []
for year in metadata:
  print(year)
  year_info = metadata[year]
  df = pd.read_excel(year_info['link'],header=year_info['header row'])
  df.columns = year_info['columns']
  df['YEAR'] = [int(year)]*df.shape[0]
  all_dfs_raw.append(df)
  if int(year) <= 2006:
    dfs_to_keep_2001_2006.append(df[['YEAR','STATE','CODE','DESCRIPTION', 'DATA TYPE', 'TOTAL']].copy())
  else:
    dfs_to_keep_2007_2018.append(df[['YEAR','STATE', 'CODE','DESCRIPTION', 'SIZE','FIRMS', 'ESTABLISHMENTS', 'EMPLOYMENT']].copy())

### First deal with the 2001-2006 data. I will pivot this data so that the different values (employment, firms, and establishments) are columns instead of taking up a whole row for each

In [ ]:
# append all the 2001 through 2006 data together
df_2001_2006 = pd.concat(dfs_to_keep_2001_2006).reset_index(drop=True)
# append all the 2007 thorugh 2018 data together
df_2007_2018 = pd.concat(dfs_to_keep_2007_2018).reset_index(drop=True)

In [ ]:
# I like the representation of having FIRMS, ESTABLISHMENTS, and EMPLOYMENT as their own columns so I will make the 2001 thorugh 2006 data match that representation
df_2001_2006 = df_2001_2006.pivot(index=['STATE','DESCRIPTION','YEAR'], columns='DATA TYPE', values='TOTAL').reset_index()

In [ ]:
# we don't need all the variables
df_2001_2006 = df_2001_2006[['STATE','DESCRIPTION','YEAR','Employment','Establishments','Firms']].copy()

In [ ]:
df_2001_2006.head()

In [ ]:
df_2001_2006.isnull().sum()

## Now deal with the 2007-2018 data. For these I only want to keep the "total" and get rid of the other breakdowns such as 0-4, 5-9, ...500+.

In [ ]:
# first I need to inspect the unique ways that establishment size have been written out over the years. It looks like I just need to search for 'total' being in the name (as lower case)
df_2007_2018['SIZE'].unique()

In [ ]:
# I will now make a new column that determines if 'total' is in the lower case string for each value in the SIZE column. Then I will subset based on that
df_2007_2018['Total Size'] = df_2007_2018['SIZE'].str.lower().str.contains('total')
# some of the years of data had a few blank rows below the header which results in 16 n/a values in this 'Total Size' column (they are n/a everywhere else too) so I will make those False too so they go away
df_2007_2018['Total Size'] = df_2007_2018['Total Size'].fillna(False)
# subset to where the row is for total and only keep a subset of the columns
df_2007_2018 = df_2007_2018[df_2007_2018['Total Size']][['YEAR','STATE','DESCRIPTION','FIRMS','ESTABLISHMENTS','EMPLOYMENT']]

In [ ]:
# now the column names need to be consistent with the column names for the 2001-2006 data
df_2007_2018.columns = ['YEAR','STATE','DESCRIPTION','Firms','Establishments','Employment']

In [ ]:
df_2007_2018.head()

In [ ]:
df_2007_2018.isnull().sum()

## Concatenate the dataframes to get a dataframe with years 2001-2018 of data

In [ ]:
# concatenate the two sets of years of data
all_years_df = pd.concat([df_2001_2006, df_2007_2018]).reset_index(drop=True)

In [ ]:
# some of the data types are just objects and that's because the data has random letters in the numeric fields for some of the rows
# I think these should be replaced with n/a's
all_years_df.dtypes

In [ ]:
# this function will find all the non numeric values in a column and then replace them with n/a so that the field can become numeric
def replace_non_numeric(df):
  non_numeric = []
  for col in df.columns:
    for i in range(df.shape[0]):
      try:
        float(df[col].iloc[i]) 
      except:
        non_numeric.append(df[col].iloc[i])
  return df.replace(np.unique(non_numeric),np.nan)

In [ ]:
# the columns that should be numeric are Employment, Establishments, and Firms
all_years_df[['Establishments','Firms','Employment']] = replace_non_numeric(all_years_df[['Establishments','Firms','Employment']].copy())

In [ ]:
all_years_df

# Classification Tagging

In [ ]:
nlp = spacy.load("en_core_web_trf") 

def lemmatize(description) -> list():

    for token in doc:
        token_lemma_list.append(token.lemma_.lower())

    return token_lemma_list


description_lemma_dict = dict()

#Iterate through unique description, get lemma and lower description add to dict
unique_descriptions = df_complete["DESCRIPTION"].unique()


for description in tqdm(unique_descriptions):
    #Turns description in to an interable token list with attrs such as lemma
    doc = nlp(description)
    
    #Keeps trak of current tokens already lemmatized and lower
    token_lemma_list = list()
    
    #Iterate through each token in doc
    for token in doc:
        
        token_lemma_list.append(token.lemma_.lower())
    
    description_lemma_dict.update({description:" ".join(token_lemma_list)})

all_years_df["Description_Lemma_Lower"] = all_years_df["DESCRIPTION"].apply(lambda description: description_lemma_dict[description])
all_years_df["Description_Lower"] = all_years_df["DESCRIPTION"].str.lower()



In [ ]:
# now add the classifications to the dataframe
def assign_classification(tokens,industry_words,commerical_words):
    try:
        for token in tokens.split(" "):

            for ind_token in industry_words:

                if ind_token in token:

                    return "IND"

            for com_token in commerical_words:

                    if com_token in token:
                        
                        return "COM"
        
        return "OTH"

    except:
        return "Not Found"


In [ ]:
# Words to classification description as IND
industry_tokens = ["mining","mill","manufacture","industrial","material","chemical","part","unit","machine","manufacturing"
                    ,"fuel","water","electric","generate","quarry","industrial","facil","equipment","metal","freight"
                ]


# Words to classification description as COM
commerical_tokens = ["service","sale","admin","office","wholesaler","financ"
                    ,"store","clothing","support","salon","broadcast","intermediat","agency","dealer"
                    ,"appraiser","telecommunication","sell","bank","repair","school","commercial","merchandise", "professional"
                    ,"advertis","publisher","agency","contractor","research","real","estate","social","markets","information","insurance","trade"
                    ,"hospitals","rental","construction"]



all_years_df["Classification_Lower"] = (all_years_df["Description_Lower"]
                                 .apply(lambda description: assign_classification(description,industry_tokens,commerical_tokens))
                                 
                                 )

all_years_df["Classification_Lemma_Lower"] = (all_years_df["Description_Lemma_Lower"]
                                 .apply(lambda description: assign_classification(description,industry_tokens,commerical_tokens))

                                    )



_ = (all_years_df[["Classification_Lower","Classification_Lemma_Lower","STATE"]].groupby('Classification_Lower').count()
.rename(columns={"STATE":"Count"})
.sort_values("Count",ascending=False)
).plot(kind="bar",title="Classification Count - Lower")

In [ ]:

# Most frequently used words in OTH description

description_list = all_years_df.query('Classification_Lemma_Lower == "OTH"')["Description_Lemma_Lower"].to_list()

count_dictionary = dict()


for description in description_list:

    for token in description.split(" "):

        if token.strip() in count_dictionary.keys():

            count_dictionary[token] += 1

        else:

            count_dictionary[token] = 1


pd.DataFrame([count_dictionary]).T.rename(columns={0:"Count"}).sort_values("Count",ascending=False).head(50)

### Add state abbreviations

In [ ]:
state_abbrev = []
for s in all_years_df['STATE']:
  if s != 'United States':
    state_object = us.states.lookup(s)
    state_abbrev.append(state_object.abbr)
  else:
    state_abbrev.append(np.nan)
all_years_df['STATE'] = state_abbrev

In [ ]:
all_years_df.isnull().sum()

In [ ]:
# there are a lot of places where STATE is null because the value was total for the whole United STates so these should be removed
all_years_df = all_years_df[~all_years_df['STATE'].isnull()].copy()

In [ ]:
all_years_df.isnull().sum()

In [ ]:
# now save this dataframe
all_years_df.to_csv('/content/drive/Shareddrives/Data606_Energy/data/fill_ready/SUSB_data_all_2001_2018.csv', index=False)

In [ ]:
import plotly.express as px

fig = px.bar(df,x="Classification",y="count",text="tokens",title="Tokens Search for to Determine Classification")

fig.update_layout(
    autosize=False,
    width=2000,
    height=1000,
)

